In [ ]:
n_NS = len(genome)
range_n_p2 = np.arange(2000, 10001, 1000)
range_n_r2 = np.arange(10, 101, 10)

arr_pbound = np.zeros((len(range_n_p2), len(range_n_r2)))
for i, n_p in enumerate(range_n_p2):
    for j, n_r in enumerate(range_n_r2):
        pbound = tregs.simulate.simrep_pbound(rnap_site, rep_site, rnap_emat, rep_emat, n_p, n_r, n_NS,
                                              ep_wt=-5, er_wt=-15)
        arr_pbound[i,j] = pbound
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
ax1, ax2, ax3 = axes

ax1.plot(range_n_p, l_pbound1)
ax1.axvspan(2000, 10000, alpha=0.7, color='lightgray')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel('Number of RNAPs')
ax1.set_ylabel('Probabilty of RNAP binding')
ax1.text(2000, max(l_pbound1) * 1.5, 'Physiological range\nof copy numbers', fontsize = 11, color = 'k',
            ha='left', va='baseline')

ax2.plot(range_n_r, l_pbound2)
ax2.set_xlabel('Number of repressors')

ax3 = sns.heatmap(arr_pbound,
            xticklabels=range_n_r2,
            yticklabels=range_n_p2,
            norm=LogNorm(),
            cmap=sns.cm.rocket_r,
            )
ax3.invert_yaxis()
ax3.set_xlabel('Number of repressors')
ax3.set_ylabel('Number of RNAPs')
ax3.collections[0].colorbar.set_label('Probability of binding') 

plt.tight_layout()
plt.show()

In [ ]:
n_p_low, n_p_high = 2000, 10000 #[BNID: 108601]
n_r_low, n_r_high = 10, 50 # [BNID: 100734, 102083, 103602]
l_mut2 = []
for mut in lac_mutants:
    rv = {}
    rv['seq'] = mut
    rnap_site_mut = mut[rnap_site_start:rnap_site_end].upper()
    rep_site_mut = mut[rep_site_start:rep_site_end].upper()
    
    n_p, n_r = 4600, 10
    rv['e_rnap'] = tregs.simulate.get_d_energy(rnap_site_mut, rnap_emat, e_wt=-5)
    rv['e_rep'] = tregs.simulate.get_d_energy(rep_site_mut, rep_emat, e_wt=-15)
    rv['pbound'] = tregs.simulate.simrep_pbound(rnap_site_mut, rep_site_mut, rnap_emat, rep_emat, n_p, n_r, n_NS,
                                                  ep_wt=-5, er_wt=-15)
    
    for i in range(10):
        n_p = np.random.randint(n_p_low, n_p_high)
        n_r = np.random.randint(n_r_low, n_r_high)
        rv['P' + str(i)] = n_p
        rv['R' + str(i)] = n_r
        rv['pbound' + str(i)] = tregs.simulate.simrep_pbound(rnap_site_mut, rep_site_mut, rnap_emat, rep_emat,
                                                             n_p, n_r, n_NS,
                                                             ep_wt=-5, er_wt=-15)
    l_mut2.append(rv)

df_mut2 = pd.DataFrame.from_records(l_mut2)
fig, ax = plt.subplots(figsize=(5,5))

for i in range(10):
    ax.scatter(df_mut2.pbound, df_mut2['pbound' + str(i)],
               alpha=0.5, c=df_mut2['P' + str(i)])
ax.plot(np.arange(0, 0.71, 0.1), np.arange(0, 0.71, 0.1), color='red')